# 8. Evaluation of results

## Setup

In [1]:
%run __init__.py

In [80]:
import pandas as pd

PMC_FILE_PATH = os.path.join(NOTEBOOK_2_RESULTS_DIR, 'pmc_dataframe.pkl')

pmc_df = pd.read_pickle(PMC_FILE_PATH)

## Analyzing the categories

In [81]:
import numpy as np

categories_df = pmc_df.copy()
categories_df['subjects'].replace('', np.nan, inplace=True)
categories_df.dropna(subset=['subjects'], inplace=True)
categories_df.head(n=10)

,id,title,abstract,full_body,authors,references,subjects,text_cleaned,num_chars_text
0,3310815,Induced Release of a Plant-Defense Volatile ‘D...,Transmission of plant pathogens by insect vect...,Introduction Transmission of plant pathogens b...,Mann Rajinder S.|Ali Jared G.|Hermann Sara L.|...,Insect vector relationships with procaryotic p...,Agriculture|Crops|Pest Control|Biology|Ecology...,Introduction Transmission of plant pathogens b...,54143
1,3547067,Carbon and Nitrogen Isotopic Survey of Norther...,The development of isotopic baselines for comp...,Introduction Stable isotope analysis is an imp...,Szpak Paul|White Christine D.|Longstaffe Fred ...,Influence of diet on the distribution of carbo...,Biology|Ecology|Biogeochemistry|Paleontology|P...,Introduction Stable isotope analysis is an imp...,74402
3,3672096,Emissions of CH4 and N2O under Different Tilla...,Understanding greenhouse gases (GHG) emissions...,Introduction With the current rise in global t...,Zhang Hai-Lin|Bai Xiao-Lin|Xue Jian-Fu|Chen Zh...,Simulation of fluxes of greenhouse gases from ...,Agriculture|Agricultural Biotechnology|Agricul...,Introduction With the current rise in global t...,29852
7,3904951,Diversity and Spatial Structure of Belowground...,Plant–mycorrhizal fungal interactions are ubiq...,Introduction More than 90% of wild terrestrial...,Toju Hirokazu|Sato Hirotoshi|Tanabe Akifumi S.,Mycorrhizal associations and other means of nu...,Agriculture|Forestry|Biology|Ecology|Community...,Introduction More than 90% of wild terrestrial...,57644
8,3951315,Effects of Introduced and Indigenous Viruses o...,The ever increasing movement of viruses around...,Introduction Long before plants were first dom...,Vincent Stuart J.|Coutts Brenda A.|Jones Roger...,Coevolution of viruses with hosts and vectors ...,Agriculture|Agroecology|Crops|Sustainable Agri...,Introduction Long before plants were first dom...,72432
9,4067337,Effects of Winter Cover Crops Residue Returnin...,Residue management in cropping systems is usef...,Introduction Enzymes play an important role in...,Hai-Ming Tang|Xiao-Ping Xiao|Wen-Guang Tang|Ye...,Enzyme activities in the rhizosphere of pepper...,Biology and Life Sciences|Agriculture|Agroecol...,Introduction Enzymes play an important role in...,27649
10,4112640,H2O2 mediates the crosstalk of brassinosteroid...,SummaryBrassinosteroids induce H2O2 accumulati...,Introduction Plants continuously face a myriad...,Zhou Jie|Wang Jian|Li Xin|Xia Xiao-Jian|Zhou Y...,The influence of 24-epibrassinolide on the hor...,Research Paper,Introduction Plants continuously face a myriad...,41814
11,4128675,Involvement of Potato (Solanum tuberosum L.) M...,Mitogen-activated protein kinase (MAPK) cascad...,Introduction Mitogen-activated protein kinase ...,Lazar Ana|Coll Anna|Dobnik David|Baebler Špela...,MAPK cascades in plant disease resistance sign...,Biology and Life Sciences|Biochemistry|Hormone...,Introduction Mitogen-activated protein kinase ...,54503
12,4171492,Multiple Different Defense Mechanisms Are Acti...,Previously described transgenic tobacco lines ...,Introduction Viruses are obligate intracellula...,Jada Balaji|Soitamo Arto J.|Siddiqui Shahid As...,Host factors in positive-strand RNA virus geno...,Biology and Life Sciences|Microbiology|Medical...,Introduction Viruses are obligate intracellula...,59734
13,4237146,Potato virus Y infection hinders potato defenc...,"In the field, plants are challenged by more th...",Introduction Plant pathogens and pests are the...,Petek Marko|Rotter Ana|Kogovšek Polona|Baebler...,Colorado potato beetle resistance to insectici...,Original Articles,Introduction Plant pathogens and pests are the...,45594


In [82]:
articles_categories = {aid: categories.split('|') 
                       for aid, categories in zip(categories_df['id'].values,
                                                  categories_df['subjects'].values)}
articles_categories['6394436']

['Review Article']

In [83]:
stop_categories = set(["research", "research paper",
                       "review article", "research articles",
                       "regular papers", "basic research papers",
                       "original paper", "research papers",
                       "primary research articles", "primary research article",
                       "original article", "original articles", "original research"])

filtered_articles_categories = {
    k: set([el for el in v if el.lower() not in stop_categories
        and not el.isnumeric()])
    for k, v in articles_categories.items()
}

final_articles_categories = {
    k: v
    for k, v in filtered_articles_categories.items()
    if len(v) != 0
}

len(final_articles_categories)

38

In [84]:
final_articles_categories

{'3310815': {'Agriculture',
  'Biology',
  'Crops',
  'Ecology',
  'Pest Control',
  'Plant Science',
  'Zoology'},
 '3547067': {'Anthropology',
  'Archaeology',
  'Archaeometry',
  'Biogeochemistry',
  'Biology',
  'Chemistry',
  'Earth Sciences',
  'Ecology',
  'Geochemistry',
  'Isotopes',
  'Paleobotany',
  'Paleontology',
  'Plant Science',
  'Radiochemistry',
  'Social and Behavioral Sciences'},
 '3672096': {'Agricultural Biotechnology',
  'Agricultural Production',
  'Agriculture',
  'Agroecology',
  'Atmospheric Science',
  'Biology',
  'Climate Change',
  'Climatology',
  'Earth Sciences',
  'Ecology',
  'Ecosystems Agroecology',
  'No-Till Farming',
  'Soil Ecology',
  'Soil Science',
  'Sustainable Agriculture'},
 '3904951': {'Agriculture',
  'Biodiversity',
  'Biology',
  'Botany',
  'Community Ecology',
  'Ecological Environments',
  'Ecology',
  'Forestry',
  'Fungal Classification',
  'Fungi',
  'Microbiology',
  'Mycology',
  'Plant Ecology',
  'Plant Science',
  'Popul

## Evaluation

* Load CSV with inferred topics from notebook 7.
* Get subset of topics for articles with subjects.
* Define function to compare predictions and true labels
* Use function to optimize parameters with grid search

In [86]:
categories_df.set_index('id', inplace=True)

articles_keys = final_articles_categories.keys()
X = categories_df.loc[articles_keys]['text_cleaned'].values

In [ ]:
y_base = final_articles_categories.values()
y_pred = final_pipe.transform(X)

In [ ]:
def get_score(topics_base, topics_pred):
    pass

## Similarity

In [173]:
from nltk.corpus import wordnet as wn

a = wn.synsets('chemistry')[0]
b = wn.synsets('chemistry')[0]
bed = wn.synsets('bed')[0]

In [174]:
a.lch_similarity(b)

3.6375861597263857

In [130]:
wn.synsets("community_ecology")

[]

## Saving the results